Voici les librairies utile pour le bon fonctionnement du code :

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk
nltk.download('punkt')

from sklearn.decomposition import PCA

#Initialising model instance
pca=PCA(.95)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Preprocessing:

In [ ]:
stemming = PorterStemmer()
nltk.download('stopwords')
stops = set(stopwords.words("english"))
def Cremove_stops(row):
    my_list = row['comment']
    meaningful_words = [w for w in my_list if not w in stops]
    return (meaningful_words)

def Cstem_list(row):
    my_list = row['comment']
    stemmed_list = [stemming.stem(word) for word in my_list]
    return (stemmed_list)

def Cidentify_tokens(row):
    review = row['comment']
    tokens = nltk.word_tokenize(review)
    # taken only words (not punctuation)
    token_words = [w for w in tokens if w.isalpha()]
    return token_words

def Premove_stops(row):
    my_list = row['parent_comment']
    meaningful_words = [w for w in my_list if not w in stops]
    return (meaningful_words)

def Pstem_list(row):
    my_list = row['parent_comment']
    stemmed_list = [stemming.stem(word) for word in my_list]
    return (stemmed_list)

def Pidentify_tokens(row):
    review = row['parent_comment']
    tokens = nltk.word_tokenize(review)
    # taken only words (not punctuation)
    token_words = [w for w in tokens if w.isalpha()]
    return token_words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


On va effectuer deux preprocessing différents, c'est pour quoi on prend de dataframes.

In [ ]:
#df = pd.read_csv("/content/drive/My Drive/NLPW/Train.csv", usecols=[1,3,4,5,6,7,9],header=0)


On applique tout sur df, pour faire un premier preprocessing.

In [29]:
dfnb = pd.read_csv("/content/drive/My Drive/NLPW/Train.csv", usecols=[1,4,5,9],header=0)

#dfnb['comment'] = dfnb.apply(Cidentify_tokens, axis=1)
#dfnb['comment'] = dfnb.apply(Cremove_stops, axis=1)
#dfnb['comment'] = dfnb.apply(Cstem_list, axis=1)

print(dfnb['comment'])


0        Well, let's be honest here, they don't actuall...
1        Well, I didn't need evidence to believe in com...
2                    Who does an "official promo" in 360p?
3                                 Grotto koth was the best
4                                         Neal's back baby
                               ...                        
14995    Well with a name like El Cubano I'm surprised ...
14996                            ... This is a good point.
14997                                                 Yep.
14998     That's what the government WANTS you to believe!
14999    because Windows 10 has the glorious start menu...
Name: comment, Length: 15000, dtype: object


In [30]:
dfnb.isnull().sum()

comment           0
parent_comment    0
score             0
label             0
dtype: int64

On garde qu'une partie de data en question:

In [31]:
df=dfnb.drop(['parent_comment','score'], axis=1) ##We could also simply load only needed columns while importing csv thanks to usecols
df.head()

,comment,label
0,"Well, let's be honest here, they don't actuall...",0
1,"Well, I didn't need evidence to believe in com...",1
2,"Who does an ""official promo"" in 360p?",0
3,Grotto koth was the best,1
4,Neal's back baby,1


In [32]:
#Preprocessing
stop_words = set(stopwords.words('english'))

df['comment'] = df['comment'].apply(lambda x: re.sub('[^a-z]', ' ', x.lower()))
df['comment'] = df['comment'].apply(lambda x: re.sub(' +', ' ', x))
df['comment'] = df['comment'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords.words('english')]))

df.head()

,comment,label
0,well let honest actually seem much moderating ...,0
1,well need evidence believe communism,1
2,official promo p,0
3,grotto koth best,1
4,neal back baby,1


In [33]:
#droping duplicates

df=df.drop_duplicates()
df=df.reset_index(drop=True)
df.describe(include='all')

,comment,label
count,14462,14462.000000
unique,14388,NaN
top,,NaN
freq,2,NaN
mean,NaN,0.506846
std,NaN,0.499970
min,NaN,0.000000
25%,NaN,0.000000
50%,NaN,1.000000
75%,NaN,1.000000


In [45]:
#Splitting training/test sets
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.25)
df_train=df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
print(df_train.shape, df_test.shape)
df_train.head()

(10846, 2) (3616, 2)


,comment,label
0,chosen one,0
1,wrong time need agree,1
2,long uses right bathroom problem,1
3,h footprints sand clannad angel beats psycho p...,0
4,killed meruem,1


Performin TF-IDF text transformation:

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
text_features_train = vectorizer.fit_transform(df_train['comment'])
text_features_train.shape

#print(text_features_train)

(10123, 13465)

Using PCA to reduce dimenstions:

In [46]:
text_features_train=text_features_train.toarray()

pca=PCA(n_components=2)
features_train = pca.fit_transform(text_features_train)
features_train.shape


(10123, 2)